## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Darts time series
from darts import TimeSeries
from darts.models import RandomForest
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mae, r2_score

# SHAP for feature importance
import shap

# Sklearn utilities
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✅ All libraries imported successfully!")

## 2. Load and Merge Data

In [ ]:
# Load all datasets
print("="*60)
print("LOADING DATASETS")
print("="*60)

# IHSG (target) - Indonesian format dd/mm/yyyy
ihsg = pd.read_csv('ihsg_daily.csv')
ihsg['Date'] = pd.to_datetime(ihsg['Date'], format='%d/%m/%Y')
ihsg.columns = ['Date', 'IHSG']
print(f"IHSG: {len(ihsg)} rows, {ihsg['Date'].min()} to {ihsg['Date'].max()}")

# STI - Indonesian format dd/mm/yyyy
sti = pd.read_csv('STI.csv')
sti['Date'] = pd.to_datetime(sti['Date'], format='%d/%m/%Y')
sti.columns = ['Date', 'STI']
print(f"STI: {len(sti)} rows, {sti['Date'].min()} to {sti['Date'].max()}")

# Commodities - US format mm/dd/yyyy
coal = pd.read_csv('Coal.csv')
coal['Date'] = pd.to_datetime(coal['Date'], format='%m/%d/%Y')
coal.columns = ['Date', 'Coal']
print(f"Coal: {len(coal)} rows, {coal['Date'].min()} to {coal['Date'].max()}")

copper = pd.read_csv('Copper.csv')
copper['Date'] = pd.to_datetime(copper['Date'], format='%m/%d/%Y')
copper.columns = ['Date', 'Copper']
print(f"Copper: {len(copper)} rows, {copper['Date'].min()} to {copper['Date'].max()}")

silver = pd.read_csv('Silver.csv')
silver['Date'] = pd.to_datetime(silver['Date'], format='%m/%d/%Y')
silver.columns = ['Date', 'Silver']
print(f"Silver: {len(silver)} rows, {silver['Date'].min()} to {silver['Date'].max()}")

tin = pd.read_csv('Tin.csv')
tin['Date'] = pd.to_datetime(tin['Date'], format='%m/%d/%Y')
tin.columns = ['Date', 'Tin']
print(f"Tin: {len(tin)} rows, {tin['Date'].min()} to {tin['Date'].max()}")

nickel = pd.read_csv('Nickel.csv')
nickel['Date'] = pd.to_datetime(nickel['Date'], format='%m/%d/%Y')
nickel.columns = ['Date', 'Nickel']
print(f"Nickel: {len(nickel)} rows, {nickel['Date'].min()} to {nickel['Date'].max()}")

In [ ]:
# Merge all datasets on Date (inner join to get common dates)
print("\n" + "="*60)
print("MERGING DATASETS")
print("="*60)

# Start with IHSG as base
df = ihsg.copy()

# Merge each dataset
for dataset, name in [(sti, 'STI'), (coal, 'Coal'), (copper, 'Copper'), 
                       (silver, 'Silver'), (tin, 'Tin'), (nickel, 'Nickel')]:
    df = df.merge(dataset, on='Date', how='inner')
    print(f"After merging {name}: {len(df)} rows")

# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

print(f"\n✅ Final merged dataset: {len(df)} rows")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
# Display merged data
print("="*60)
print("MERGED DATA PREVIEW")
print("="*60)
df.head(10)

In [ ]:
# Check data types and missing values
print("="*60)
print("DATA TYPES & MISSING VALUES")
print("="*60)
df.info()
print("\n" + "="*60)
print("MISSING VALUES PER COLUMN")
print("="*60)
print(df.isnull().sum())

In [ ]:
# Descriptive statistics
print("="*60)
print("DESCRIPTIVE STATISTICS")
print("="*60)
df.describe().round(2)

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Visualization: Time series plots
fig, axes = plt.subplots(4, 2, figsize=(16, 14))
fig.suptitle('Time Series Visualization - IHSG, STI, and Commodity Prices', fontsize=14, fontweight='bold')

# Define colors
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
columns = ['IHSG', 'STI', 'Coal', 'Copper', 'Silver', 'Tin', 'Nickel']

for idx, (col, color) in enumerate(zip(columns, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.plot(df['Date'], df[col], color=color, linewidth=0.8, alpha=0.9)
    ax.set_title(f'{col}', fontsize=12)
    ax.set_ylabel('Price/Index')
    ax.fill_between(df['Date'], df[col], alpha=0.2, color=color)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3)

# Hide the last empty subplot
axes[3, 1].axis('off')

plt.tight_layout()
plt.savefig('model2_eda_timeseries.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation Analysis
fig, ax = plt.subplots(figsize=(10, 8))

corr_matrix = df.drop('Date', axis=1).corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.3f', cmap='RdBu_r', 
            center=0, square=True, linewidths=0.5, ax=ax,
            annot_kws={'size': 10})

ax.set_title('Correlation Matrix - IHSG, STI, and Commodities', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('model2_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("CORRELATION WITH IHSG")
print("="*60)
ihsg_corr = corr_matrix['IHSG'].drop('IHSG').sort_values(ascending=False)
for var, corr in ihsg_corr.items():
    direction = "↑ Positive" if corr > 0 else "↓ Negative"
    strength = "Strong" if abs(corr) > 0.7 else "Moderate" if abs(corr) > 0.4 else "Weak"
    print(f"{var:15} : {corr:+.4f} ({strength} {direction})")

## 4. Create Darts TimeSeries Objects

In [ ]:
# Define target and covariate columns
TARGET_COL = 'IHSG'
COVARIATE_COLS = ['STI', 'Coal', 'Copper', 'Silver', 'Tin', 'Nickel']

# Set Date as index with business day frequency
df_ts = df.set_index('Date')

# Create target TimeSeries
target_series = TimeSeries.from_dataframe(
    df_ts, 
    value_cols=TARGET_COL,
    fill_missing_dates=False  # Daily data may have gaps (weekends/holidays)
)

# Create covariates TimeSeries
covariates = TimeSeries.from_dataframe(
    df_ts,
    value_cols=COVARIATE_COLS,
    fill_missing_dates=False
)

print("="*60)
print("DARTS TIMESERIES CREATED")
print("="*60)
print(f"Target Series (IHSG):")
print(f"  - Start: {target_series.start_time()}")
print(f"  - End: {target_series.end_time()}")
print(f"  - Length: {len(target_series)} time steps")
print(f"\nCovariates:")
print(f"  - Components: {covariates.components.tolist()}")
print(f"  - Length: {len(covariates)} time steps")

## 5. Train/Test Split and Scaling

In [ ]:
# Train/Test Split (80/20)
TRAIN_RATIO = 0.8
split_point = int(len(target_series) * TRAIN_RATIO)

train_target = target_series[:split_point]
test_target = target_series[split_point:]

train_cov = covariates[:split_point]
test_cov = covariates[split_point:]

# Scale the data using Darts Scaler
scaler_target = Scaler()
scaler_cov = Scaler()

# Fit on training data only to avoid data leakage
train_target_scaled = scaler_target.fit_transform(train_target)
test_target_scaled = scaler_target.transform(test_target)

train_cov_scaled = scaler_cov.fit_transform(train_cov)
test_cov_scaled = scaler_cov.transform(test_cov)

# Full scaled series for prediction
target_scaled = scaler_target.transform(target_series)
cov_scaled = scaler_cov.transform(covariates)

print("="*60)
print("TRAIN/TEST SPLIT")
print("="*60)
print(f"Train Period: {train_target.start_time()} to {train_target.end_time()} ({len(train_target)} days)")
print(f"Test Period:  {test_target.start_time()} to {test_target.end_time()} ({len(test_target)} days)")
print(f"\nTrain/Test Ratio: {TRAIN_RATIO*100:.0f}% / {(1-TRAIN_RATIO)*100:.0f}%")

## 6. Hyperparameter Tuning with GridSearch

In [ ]:
# Define hyperparameter grid for RandomForest
# For daily data, use shorter lags (days instead of months)
param_grid = {
    'lags': [5, 10, 21],                      # Target lags (days) - ~1 week, 2 weeks, 1 month
    'lags_past_covariates': [5, 10, 21],     # Covariate lags
    'n_estimators': [100, 200, 300],         # Number of trees
    'max_depth': [5, 10, None],              # Maximum tree depth
}

print("="*60)
print("HYPERPARAMETER GRID")
print("="*60)
for param, values in param_grid.items():
    print(f"{param}: {values}")

total_combinations = 1
for values in param_grid.values():
    total_combinations *= len(values)
print(f"\nTotal combinations to evaluate: {total_combinations}")

In [ ]:
# Perform GridSearch
print("="*60)
print("RUNNING GRIDSEARCH (this may take several minutes for daily data...)")
print("="*60)

best_model, best_params, best_score = RandomForest.gridsearch(
    parameters=param_grid,
    series=train_target_scaled,
    past_covariates=cov_scaled,  # Full covariates to cover validation period
    val_series=test_target_scaled,
    metric=mape,
    verbose=True,
    n_jobs=-1,  # Use all CPU cores
)

print("\n" + "="*60)
print("GRIDSEARCH RESULTS")
print("="*60)
print(f"Best MAPE Score: {best_score:.4f}%")
print(f"\nBest Hyperparameters:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

## 7. Train Best Model and Generate Predictions

In [ ]:
# Create and train the best model with optimal hyperparameters
final_model = RandomForest(
    lags=best_params['lags'],
    lags_past_covariates=best_params['lags_past_covariates'],
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    output_chunk_length=1,
    random_state=42
)

# Fit on training data
final_model.fit(train_target_scaled, past_covariates=train_cov_scaled)

# Generate predictions on test set
n_test = len(test_target_scaled)
predictions_scaled = final_model.predict(n=n_test, past_covariates=cov_scaled)

# Inverse transform to original scale
predictions = scaler_target.inverse_transform(predictions_scaled)
test_actual = scaler_target.inverse_transform(test_target_scaled)

print("="*60)
print("MODEL TRAINED AND PREDICTIONS GENERATED")
print("="*60)
print(f"Prediction Period: {predictions.start_time()} to {predictions.end_time()}")
print(f"Number of predictions: {len(predictions)} trading days")

## 8. Model Evaluation

In [ ]:
# Calculate evaluation metrics
mape_score = mape(test_actual, predictions)
rmse_score = rmse(test_actual, predictions)
mae_score = mae(test_actual, predictions)
r2 = r2_score(test_actual, predictions)

print("="*60)
print("MODEL EVALUATION METRICS")
print("="*60)
print(f"MAPE (Mean Absolute Percentage Error): {mape_score:.4f}%")
print(f"RMSE (Root Mean Square Error):         {rmse_score:.4f}")
print(f"MAE (Mean Absolute Error):             {mae_score:.4f}")
print(f"R² Score:                              {r2:.4f}")
print("="*60)

# Interpretation
print("\n📊 INTERPRETATION:")
if mape_score < 5:
    print(f"  ✅ MAPE < 5%: Excellent forecasting accuracy")
elif mape_score < 10:
    print(f"  ✅ MAPE < 10%: Good forecasting accuracy")
elif mape_score < 20:
    print(f"  ⚠️ MAPE < 20%: Reasonable forecasting accuracy")
else:
    print(f"  ❌ MAPE > 20%: Poor forecasting accuracy")

In [ ]:
# Visualization: Actual vs Predicted
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Full time series with predictions
ax1 = axes[0]
ax1.plot(train_target.time_index, train_target.values(), 
         label='Training Data', color='#1f77b4', linewidth=0.8, alpha=0.8)
ax1.plot(test_actual.time_index, test_actual.values(), 
         label='Actual (Test)', color='#2ca02c', linewidth=1.5)
ax1.plot(predictions.time_index, predictions.values(), 
         label='Predicted', color='#d62728', linewidth=1.5, linestyle='--')
ax1.axvline(x=train_target.end_time(), color='gray', linestyle=':', alpha=0.7, label='Train/Test Split')
ax1.set_title('IHSG Prediction - Random Forest Model (Daily Data)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('IHSG Index')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Test period close-up
ax2 = axes[1]
ax2.plot(test_actual.time_index, test_actual.values(), 
         label='Actual', color='#2ca02c', linewidth=1.5)
ax2.plot(predictions.time_index, predictions.values(), 
         label='Predicted', color='#d62728', linewidth=1.5, linestyle='--')
ax2.fill_between(test_actual.time_index, 
                  test_actual.values().flatten(), 
                  predictions.values().flatten(), 
                  alpha=0.3, color='gray', label='Error')
ax2.set_title(f'Test Period: Actual vs Predicted (MAPE: {mape_score:.2f}%)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('IHSG Index')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('model2_prediction_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Feature Importance with SHAP

In [ ]:
# Prepare feature matrix for SHAP
lags = best_params['lags']
lags_cov = best_params['lags_past_covariates']

# Create lagged feature dataframe
def create_lagged_features(target_df, cov_df, target_lags, cov_lags):
    """Create lagged features for SHAP analysis"""
    max_lag = max(target_lags, cov_lags)
    features = pd.DataFrame(index=target_df.index[max_lag:])
    
    # Target lags
    for lag in range(1, target_lags + 1):
        features[f'IHSG_lag{lag}'] = target_df['IHSG'].shift(lag).values[max_lag:]
    
    # Covariate lags
    for col in cov_df.columns:
        for lag in range(1, cov_lags + 1):
            features[f'{col}_lag{lag}'] = cov_df[col].shift(lag).values[max_lag:]
    
    return features.dropna()

# Prepare data
target_df = df.set_index('Date')[['IHSG']]
cov_df = df.set_index('Date')[COVARIATE_COLS]

X_features = create_lagged_features(target_df, cov_df, lags, lags_cov)

print("="*60)
print("FEATURE MATRIX FOR SHAP")
print("="*60)
print(f"Feature matrix shape: {X_features.shape}")
print(f"\nFeatures ({len(X_features.columns)}):")
for i, col in enumerate(X_features.columns, 1):
    print(f"  {i}. {col}")

In [ ]:
# Train a standalone RandomForest model for SHAP analysis

# Prepare target (shifted to align with features)
max_lag = max(lags, lags_cov)
y_target = target_df['IHSG'].values[max_lag + 1:]  # +1 for next day prediction
X_train_shap = X_features.iloc[:-1].values  # Remove last row to match y_target length

# Ensure alignment
min_len = min(len(X_train_shap), len(y_target))
X_train_shap = X_train_shap[:min_len]
y_target = y_target[:min_len]

# Train RF model for SHAP
rf_shap = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    random_state=42,
    n_jobs=-1
)
rf_shap.fit(X_train_shap, y_target)

print("="*60)
print("RANDOM FOREST FOR SHAP ANALYSIS")
print("="*60)
print(f"Model fitted on {X_train_shap.shape[0]} samples with {X_train_shap.shape[1]} features")
print(f"R² Score: {rf_shap.score(X_train_shap, y_target):.4f}")

In [ ]:
# SHAP Analysis
print("="*60)
print("COMPUTING SHAP VALUES (this may take a moment...)")
print("="*60)

# Create SHAP explainer
explainer = shap.TreeExplainer(rf_shap)

# Use a sample for SHAP (full dataset can be slow for daily data)
sample_size = min(1000, len(X_train_shap))
np.random.seed(42)
sample_idx = np.random.choice(len(X_train_shap), sample_size, replace=False)
X_sample = X_train_shap[sample_idx]

# Calculate SHAP values
shap_values = explainer.shap_values(X_sample)

# Create DataFrame with feature names
feature_names = X_features.columns[:X_train_shap.shape[1]].tolist()
X_sample_df = pd.DataFrame(X_sample, columns=feature_names)

print(f"✅ SHAP values computed on {sample_size} samples!")

In [ ]:
# SHAP Summary Plot (Bar)
fig, ax = plt.subplots(figsize=(12, 10))
shap.summary_plot(shap_values, X_sample_df, plot_type="bar", show=False, max_display=20)
plt.title('SHAP Feature Importance - Mean |SHAP Value| (Top 20)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('model2_shap_importance_bar.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# SHAP Summary Plot (Beeswarm)
fig, ax = plt.subplots(figsize=(12, 12))
shap.summary_plot(shap_values, X_sample_df, show=False, max_display=20)
plt.title('SHAP Feature Importance - Beeswarm Plot (Top 20)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('model2_shap_importance_beeswarm.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate aggregated feature importance by variable (not by lag)
feature_importance_by_var = {}

for col in X_sample_df.columns:
    # Extract variable name (remove lag suffix)
    if '_lag' in col:
        var_name = col.rsplit('_lag', 1)[0]
    else:
        var_name = col
    
    col_idx = list(X_sample_df.columns).index(col)
    importance = np.abs(shap_values[:, col_idx]).mean()
    
    if var_name not in feature_importance_by_var:
        feature_importance_by_var[var_name] = 0
    feature_importance_by_var[var_name] += importance

# Sort by importance
sorted_importance = dict(sorted(feature_importance_by_var.items(), key=lambda x: x[1], reverse=True))

print("="*60)
print("AGGREGATED FEATURE IMPORTANCE BY VARIABLE")
print("="*60)
total_importance = sum(sorted_importance.values())
for var, importance in sorted_importance.items():
    pct = (importance / total_importance) * 100
    bar = "█" * int(pct / 2)
    print(f"{var:15} : {importance:8.4f} ({pct:5.2f}%) {bar}")

In [ ]:
# Visualization: Aggregated Feature Importance
fig, ax = plt.subplots(figsize=(10, 6))

vars_names = list(sorted_importance.keys())
importances = list(sorted_importance.values())

# Color coding: IHSG = blue, STI = green, Commodities = orange
colors = []
for v in vars_names:
    if 'IHSG' in v:
        colors.append('#1f77b4')
    elif 'STI' in v:
        colors.append('#2ca02c')
    else:
        colors.append('#ff7f0e')

bars = ax.barh(vars_names[::-1], importances[::-1], color=colors[::-1])
ax.set_xlabel('Mean |SHAP Value|', fontsize=12)
ax.set_title('Feature Importance by Variable (Aggregated across lags)', fontsize=14, fontweight='bold')

# Add value labels
for bar, val in zip(bars, importances[::-1]):
    ax.text(val + max(importances)*0.01, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', va='center', fontsize=10)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='Target (IHSG)'),
    Patch(facecolor='#2ca02c', label='Regional Index (STI)'),
    Patch(facecolor='#ff7f0e', label='Commodities')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('model2_feature_importance_aggregated.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Results Summary

In [ ]:
# Final Summary
print("="*70)
print("HASIL PENELITIAN MODEL 2: IHSG DENGAN KOMODITAS DAN STI")
print("="*70)

print("\n📊 DATA:")
print(f"   • Periode Data     : {df['Date'].min().strftime('%d %B %Y')} - {df['Date'].max().strftime('%d %B %Y')}")
print(f"   • Total Observasi  : {len(df)} hari trading")
print(f"   • Train/Test Split : {TRAIN_RATIO*100:.0f}% / {(1-TRAIN_RATIO)*100:.0f}%")

print("\n🎯 TARGET:")
print(f"   • Variabel Target  : IHSG (Indeks Harga Saham Gabungan) - Daily")

print("\n📈 COVARIATES:")
print("   Regional Index:")
print("     1. STI (Straits Times Index)")
print("   Commodities:")
for i, col in enumerate(['Coal', 'Copper', 'Silver', 'Tin', 'Nickel'], 2):
    print(f"     {i}. {col}")

print("\n⚙️ BEST HYPERPARAMETERS (via GridSearch):")
for param, value in best_params.items():
    print(f"   • {param}: {value}")

print("\n📏 MODEL PERFORMANCE:")
print(f"   • MAPE  : {mape_score:.4f}%")
print(f"   • RMSE  : {rmse_score:.4f}")
print(f"   • MAE   : {mae_score:.4f}")
print(f"   • R²    : {r2:.4f}")

print("\n🔍 FEATURE IMPORTANCE (SHAP - Top 3):")
for i, (var, imp) in enumerate(list(sorted_importance.items())[:3], 1):
    pct = (imp / total_importance) * 100
    print(f"   {i}. {var}: {pct:.2f}%")

print("\n" + "="*70)
print("Source: Author's calculation, 2025")
print("="*70)

In [ ]:
# Save results to CSV
results_df = pd.DataFrame({
    'Date': test_actual.time_index,
    'Actual_IHSG': test_actual.values().flatten(),
    'Predicted_IHSG': predictions.values().flatten(),
    'Error': (predictions.values().flatten() - test_actual.values().flatten()),
    'APE_%': np.abs((predictions.values().flatten() - test_actual.values().flatten()) / test_actual.values().flatten()) * 100
})

results_df.to_csv('model2_predictions.csv', index=False)
print("✅ Predictions saved to 'model2_predictions.csv'")

# Also save the merged dataset
df.to_csv('model2_merged_data.csv', index=False)
print("✅ Merged dataset saved to 'model2_merged_data.csv'")

# Display prediction results
print("\nPrediction Results (first 10 rows):")
results_df.head(10).round(2)